### This code is designed to take the CSTDM OD data from Madhu and site early stations
It uses a weighted k-means cluster to take the origin data and cluster them into centroids which represent station locations. Makes a map and js file of the results

Weight (i.e. trips) that is assigned to clusters would be used to partition statewide fuel demands to different stations
- could change the weight of some clusters by eliminating some of the origins that are far from the centroid (i.e. trucks that wouldn't be captured by the station bc too far away) in order to more realistically partition demands.
- could take really large cluster locations with alot of captured truck trips and split into multiple smaller locations (or maybe that's just combining outputs of runs with smaller and larger k values)
- 


In [1]:
import folium
import pandas as pd
import numpy as np
import json

#ChatGPT helped me write this weighted kmeans function
def weighted_kmeans(points, weights, k, max_iters=1000):
    # Randomly initialize centroids
    centroids = points[np.random.choice(points.shape[0], k, replace=False)]
    
    for _ in range(max_iters):
        # Assignment step
        distances = np.linalg.norm(points[:, np.newaxis] - centroids, axis=2)
        weighted_distances = distances / weights[:, np.newaxis]
        cluster_assignments = np.argmin(weighted_distances, axis=1)

        # Update step
        new_centroids = np.zeros((k, points.shape[1]))
        for i in range(k):
            assigned_points = points[cluster_assignments == i]
            assigned_weights = weights[cluster_assignments == i]
            if len(assigned_points) > 0:
                new_centroids[i] = np.average(assigned_points, weights=assigned_weights, axis=0)
        
        centroids = new_centroids

    return centroids, cluster_assignments
    
#TAZ coordinates
tazdf=pd.read_csv("tazListI.csv")

#Truck Origins
truckorigindf=pd.read_csv("TAZ_Origin.csv")

#Importing the Annual Origin & Desination Data Trips from 2020 Data for Heavy Trucks
O_D_trucking_data = pd.read_csv("TRIPS_TRUCK_ANNUAL-selected/TRIPS_TRUCK_ANNUAL_TAZ_HEAVY_2020.csv",  header=None, index_col=False)
O_D_trucking_data.columns = ["origin", "destination", "annual_truck_trips"]

In [2]:
# merge dataframes to add TAZ coordinates
truckorigindf=truckorigindf.merge(tazdf[['TAZ', 'Long', 'Lat']], left_on='TAZ_Zone', right_on='TAZ', how='left')
truckorigindf

,County,TAZ_Zone,TAZ_Int,Trip_Heavy,Trip_Light,Trip_Medium1,Trip_Medium2,TAZ,Long,Lat
0,Del Norte,100,100,9.16,0.16,1.03,1.27,100,-124.218478,41.788927
1,Del Norte,101,101,10.78,0.18,1.12,1.38,101,-124.168790,41.771830
2,Del Norte,102,102,38.37,0.99,4.99,6.09,102,-124.156931,41.840055
3,Del Norte,103,103,79.62,1.53,9.74,12.61,103,-123.880232,41.898154
4,Del Norte,104,104,1.93,0.00,0.20,0.20,104,-123.876849,41.615444
...,...,...,...,...,...,...,...,...,...,...
5449,Los Angeles,3599,3599,0.15,0.00,0.04,0.04,3599,-118.800024,34.793284
5450,Riverside,5450,5450,14.06,0.44,3.65,4.87,5450,-117.500450,33.915658
5451,Orange,5797,5797,16.65,0.69,5.84,7.27,5797,-117.656689,33.792941
5452,Santa Cruz,3114,3114,6.31,0.16,1.38,1.74,3114,-121.865501,37.022799


In [3]:
# Step 1: Rename columns for merge clarity
tazdf = tazdf.rename(columns={"TAZ": "origin"})  # temporary for merge
merged_df = O_D_trucking_data.merge(tazdf[["origin", "Lat", "Long"]], on="origin", how="left")
merged_df = merged_df.rename(columns={"Lat": "origin_lat", "Long": "origin_long"})

tazdf = tazdf.rename(columns={"origin": "destination"})  # change key for next merge
merged_df = merged_df.merge(tazdf[["destination", "Lat", "Long"]], on="destination", how="left")
merged_df = merged_df.rename(columns={"Lat": "destination_lat", "Long": "destination_long"})

# Step 2: Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + \
        np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# Step 3: Apply the distance calculation
merged_df["distance_km"] = haversine(
    merged_df["origin_lat"], merged_df["origin_long"],
    merged_df["destination_lat"], merged_df["destination_long"]
)

In [4]:
merged_df

,origin,destination,annual_truck_trips,origin_lat,origin_long,destination_lat,destination_long,distance_km
0,100,100,0.20,41.788927,-124.218478,41.788927,-124.218478,0.000000
1,100,101,0.09,41.788927,-124.218478,41.771830,-124.168790,4.537515
2,100,102,0.26,41.788927,-124.218478,41.840055,-124.156931,7.637934
3,100,103,0.02,41.788927,-124.218478,41.898154,-123.880232,30.538297
4,100,105,0.13,41.788927,-124.218478,40.856361,-124.150698,103.850958
...,...,...,...,...,...,...,...,...
2219321,6910,6903,0.03,33.031960,-117.124727,32.876636,-117.207025,18.901292
2219322,6910,6904,0.26,33.031960,-117.124727,32.885741,-117.179226,17.035355
2219323,6910,6905,0.46,33.031960,-117.124727,32.892994,-117.156861,15.740450
2219324,6910,6906,0.01,33.031960,-117.124727,32.827140,-117.122195,22.776171


In [5]:
# Logic Check to Make Sure the Annual Trips Are Adding Up

truck_trips_by_origin = merged_df.groupby("origin")["annual_truck_trips"].sum().reset_index()
truck_trips_by_origin

,origin,annual_truck_trips
0,100,9.36
1,101,10.78
2,102,38.37
3,103,79.62
4,104,1.93
...,...,...
5532,6906,7.48
5533,6907,1.63
5534,6908,2.72
5535,6909,21.50


In [6]:
Total_Number_Stations_TTM_data = pd.read_csv("Data for the Clusters/Demand_Projections_for_SERA(Total Number of Stations (TTM)).csv", header = 1)
Total_Number_Stations_TTM_data.rename(columns = {'Unnamed: 0' : 'Year'}, inplace = True)

# The .1 names are the high fuel scenario while the one that is normally named is the low fuel scenario
Total_Number_Stations_TTM_data


,Year,MHDV Demand (tons/day) of Gasoline Equivalent,Average New Station Size,New Stations Needed,Cumulative Number of Stations,Average Utilization,Demand Per Station (Assuming each station gets an equal demand),MHDV Demand (tons/day) of Gasoline Equivalent.1,Average New Station Size .1,New Stations Needed.1,Cumulative Number of Stations.1,Average Utilization.1,Demand Per Station (Assuming each station gets an equal demand).1
0,2024,0.636486,4.300000,2,2,0.07,0.32,0.64,4.456856,1,1,0.142810,0.64
1,2025,4.405447,4.500000,2,4,0.25,1.10,8.23,4.902542,3,4,0.429444,2.06
2,2026,14.956395,4.900000,4,8,0.40,1.87,30.60,5.392796,7,11,0.537737,2.78
3,2027,32.418607,5.400000,5,13,0.51,2.49,67.81,5.932076,9,20,0.614755,3.39
4,2028,56.503797,5.900000,6,19,0.57,2.97,119.03,6.525283,11,31,0.653711,3.84
5,2029,86.601304,6.500000,7,26,0.59,3.33,183.03,7.177812,13,44,0.664603,4.16
6,2030,122.114870,7.200000,7,33,0.62,3.70,258.63,7.895593,13,57,0.684141,4.54
7,2031,168.521119,7.600000,9,42,0.64,4.01,358.84,8.290372,16,73,0.702662,4.92
8,2032,225.795458,7.900000,10,52,0.66,4.34,483.34,8.704891,19,92,0.714924,5.25
9,2033,293.503573,8.300000,12,64,0.66,4.59,631.37,9.140136,22,114,0.719788,5.54


In [7]:
Total_Truck_Stock_TTM_data = pd.read_csv("Data for the Clusters/Demand_Projections_for_SERA(Total Truck Stocks (TTM)).csv", header = 1)
Total_Truck_Stock_TTM_data.rename(columns = {'Thousands of Trucks' : 'Year'}, inplace = True)
Total_Truck_Stock_TTM_data.drop(columns = ['Unnamed: 13', 'Unnamed: 14'], inplace = True)
# The .1 names are the high fuel scenario while the one that is normally named is the low fuel scenario
Total_Truck_Stock_TTM_data

,Year,Short-Haul,Long-Haul,Short-Haul (Actual Values),Long-Haul (Actual Values),Total Possible Demand (tonnes/day) (LH trucks use 40 kg/day and the SH trucks use 25 kg/day),Assuming an 8 tonne station size,Short-Haul.1,Long-Haul.1,Short-Haul (Actual Values).1,Long-Haul (Actual Values).1,Total Possible Demand (LH trucks use 40 kg/day and the SH trucks use 25 kg/day),Assuming an 8 tonne station size.1
0,2024,0.0,0.0,0,0,0.000,0,0.0,0.0,0,0,0.000,0
1,2025,0.0,0.1,29,60,3.125,0,0.2,0.0,188,38,6.220,1
2,2026,0.1,0.3,114,314,15.410,2,0.7,0.2,659,245,26.275,3
3,2027,0.3,0.8,256,765,37.000,5,1.4,0.6,1417,618,60.145,8
4,2028,0.5,1.4,457,1414,67.985,8,2.5,1.2,2460,1168,108.220,14
5,2029,0.7,2.3,712,2258,108.120,14,3.8,1.9,3786,1879,169.810,21
6,2030,1.0,3.3,1018,3297,157.330,20,5.4,2.7,5394,2745,244.650,31
7,2031,1.4,4.7,1395,4679,222.035,28,7.7,3.8,7653,3788,342.845,43
8,2032,1.8,6.4,1846,6417,302.830,38,10.6,5.0,10590,5022,465.630,58
9,2033,2.4,8.5,2377,8523,400.345,50,14.2,6.5,14227,6456,613.915,77


In [8]:
# Assuming fuel efficiency is 7.11 miles/kg of Hydrogen
# 11.44 km/kg of Hydrogen

# Assuming that the annual trucks trips are 1000s

# If all of the trucks were turned into hydrogen

merged_df["annual_kg_demand"] = merged_df["annual_truck_trips"] * merged_df["distance_km"] / 11.44 * 1000/365
merged_df.rename(columns = {'origin_lat':'Lat', 'origin_long': 'Long'}, inplace= True)

merged_df.head()

,origin,destination,annual_truck_trips,Lat,Long,destination_lat,destination_long,distance_km,annual_kg_demand
0,100,100,0.20,41.788927,-124.218478,41.788927,-124.218478,0.000000,0.000000
1,100,101,0.09,41.788927,-124.218478,41.771830,-124.168790,4.537515,0.097801
2,100,102,0.26,41.788927,-124.218478,41.840055,-124.156931,7.637934,0.475587
3,100,103,0.02,41.788927,-124.218478,41.898154,-123.880232,30.538297,0.146270
4,100,105,0.13,41.788927,-124.218478,40.856361,-124.150698,103.850958,3.233218


In [9]:
# Sum annual_kg_demand by origin
agg_df = merged_df.groupby('origin', as_index=False)['annual_kg_demand'].sum()

# Get unique coordinates for each origin
coord_df = merged_df[['origin', 'Lat', 'Long']].drop_duplicates(subset='origin')

# Merge them back
result_df = pd.merge(agg_df, coord_df, on='origin', how='left')
result_df

,origin,annual_kg_demand,Lat,Long
0,100,679.384373,41.788927,-124.218478
1,101,823.581572,41.771830,-124.168790
2,102,2428.873617,41.840055,-124.156931
3,103,8143.326457,41.898154,-123.880232
4,104,32.343072,41.615444,-123.876849
...,...,...,...,...
5532,6906,24.114554,32.827140,-117.122195
5533,6907,7.872110,32.881281,-117.233804
5534,6908,13.016950,32.878081,-117.223180
5535,6909,87.124262,32.917119,-117.181820


In [10]:
truckingorigindf = result_df

In [11]:

truckingorigindf

,origin,annual_kg_demand,Lat,Long
0,100,679.384373,41.788927,-124.218478
1,101,823.581572,41.771830,-124.168790
2,102,2428.873617,41.840055,-124.156931
3,103,8143.326457,41.898154,-123.880232
4,104,32.343072,41.615444,-123.876849
...,...,...,...,...
5532,6906,24.114554,32.827140,-117.122195
5533,6907,7.872110,32.881281,-117.233804
5534,6908,13.016950,32.878081,-117.223180
5535,6909,87.124262,32.917119,-117.181820


In [12]:
#This parameter adds weighting to the kmeans, i.e. 
# TAZs with more trips will pull cluster centroids closer
weighting_factor='annual_kg_demand'

# Convert DataFrame to NumPy array for processing
points = truckingorigindf[['Long', 'Lat']].values
weights = truckingorigindf[weighting_factor].values

valid_rows = weights > 0
points = points[valid_rows]
weights = weights[valid_rows]

# Filter the original DataFrame to match
truckingorigindf = truckingorigindf[valid_rows].copy()


# Number of clusters
k = 147

centroids, assignments = weighted_kmeans(points, weights, k,10000)

# Add cluster assignments to the original DataFrame
truckingorigindf['cluster'] = assignments

k_meansdf = pd.DataFrame(centroids, columns=['Long', 'Lat'])
k_meansdf['cluster'] = k_meansdf.index 
# display(truckorigindf.head())

# sum weights for each cluster to mark size of cluster into new dataframe
cluster_weights = truckingorigindf.groupby('cluster')[weighting_factor].sum().reset_index()

# Merge the summed weights into the centroids DataFrame
k_meansdf = k_meansdf.merge(cluster_weights, on='cluster', how='left')
display(k_meansdf)

#folium make map
mapcenter={'lat':36.8268747428773, 'lng': -120.12451171875001}
map = folium.Map(location=[mapcenter['lat'], mapcenter['lng']], zoom_start=6)

# Add circle markers for all truck origins 
# (size and opacity of circle is a function of # of trips)
for _, row in truckingorigindf.iterrows():
    demand = row['annual_kg_demand']
    radius = np.log10(demand + 1) * 2 #np.log10(demand + 1) * 2  # +1 avoids log(0)
    
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius= radius,
        #row['annual_kg_demand']**.4,  
        popup=f"TAZ: {row['origin']}<br>Long: {row['Long']}<br>Lat: {row['Lat']}<br>Cluster: {row['cluster']}",
        color=None,  
        fill=True,
        fill_color='blue',  
        fill_opacity=0.2
    ).add_to(map)

# Add circle markers for clusters (i.e. stations)
# radius of circle is function of number of trips closest to that station
for _, row in k_meansdf.iterrows():
    demand = row['annual_kg_demand']
    radius = np.log10(demand + 1) * 2  # +1 avoids log(0)
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius= radius,
        #row['annual_kg_demand']**.5/10,  
        popup=f"Cluster: {row['cluster']}<br>Long: {row['Long']}<br>Lat: {row['Lat']}<br>Weight:{row['annual_kg_demand']}",
        color='black',  
        weight=2,
        fill=True,
        fill_color='red',  
        fill_opacity=0.6  
    ).add_to(map)

# Save the map to an HTML file
map.save('Data for the Clusters/taz_map_'+str(k)+'clusters.html')

#write output of json file
#truckingorigindf.drop(columns= ['origin', 'Lat', 'Long', 'annual_kg_demand'], inplace = True)
                   
#['County','TAZ_Zone','TAZ_Int', 'Trip_Light','Trip_Medium1','Trip_Medium2'], inplace=True)
jsontruckstring=json.dumps(truckingorigindf.to_dict(orient='records'))
jsonclusterstring=json.dumps(k_meansdf.to_dict(orient='records'))

with open('Data for the Clusters/tazdata_'+str(k)+'clusters.js', 'w') as file:
    file.write("tazdata="+jsontruckstring+";  "+"clusterdata="+jsonclusterstring+";")

# Export truck origin data with cluster assignments to CSV
truckingorigindf.to_csv('Data for the Clusters/tazdata_'+str(k)+'clusters.csv', index=False)

# Export cluster data to CSV
k_meansdf.to_csv('Data for the Clusters/clusterdata_'+str(k)+'clusters.csv', index=False)

,Long,Lat,cluster,annual_kg_demand
0,-122.039941,37.324443,0,24661.998860
1,-118.929036,37.734908,1,21471.367662
2,-122.401786,37.789198,2,3549.637014
3,-118.258853,33.906761,3,20971.176595
4,-123.677235,41.229668,4,50690.596150
...,...,...,...,...
142,-119.571517,36.568172,142,119166.279613
143,-122.435701,37.655244,143,7754.116683
144,-122.228550,38.308683,144,25657.796518
145,-117.914950,33.714459,145,9341.594356


In [13]:
# Include the short haul number of trucks into it
# To also determine demand
#Madhus data trip_heavy does not differentiate between SH and LH heavy-duty truckss
# Ask Madhu about which O-D file to use? Is it the processed files
# The way we are calculating the clustering does not determine the way trucks are driving

In [14]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Step 1: Load your 147-cluster data
clusters = pd.read_csv("Data for the Clusters/clusterdata_147clusters.csv")

# Step 2: Define fixed coverage stations (starting in 2025)
coverage_data = {
    'Name': ['Oakland', 'Fowler', 'Orland', 'Bakersfield', 'Lost Hills',
             'Mojave', 'Los Angeles', 'Santa Nella', 'Coachella', 'W. Sacramento'],
    'latitude': [37.748749, 36.605293, 39.740147, 35.060661, 35.615117,
                 35.002833, 33.991456, 37.110064, 33.713731, 38.575739],
    'longitude': [-122.191906, -119.658529, -122.203058, -118.969539, -119.657112,
                  -118.158635, -118.159969, -121.014661, -116.175475, -121.575826],
    'year_built': [2025]*10
}
coverage_df = pd.DataFrame(coverage_data)

# Step 3: Assign each coverage station to its nearest cluster for demand
# Step 3 (FIXED): Assign demand from nearest cluster
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(clusters[['Lat', 'Long']])

# Fix the column names for querying
coverage_query = coverage_df.rename(columns={'latitude': 'Lat', 'longitude': 'Long'})
distances, indices = nn.kneighbors(coverage_query[['Lat', 'Long']])

coverage_df['cluster'] = clusters.iloc[indices.flatten()]['cluster'].values
coverage_df['annual_kg_demand'] = clusters.iloc[indices.flatten()]['annual_kg_demand'].values

# Step 4: Define rollout plan (MHDV tons/day gasoline equiv and station counts)
mhdv_rollout = {
    #2024: (0.636485662, 2),
    2025: (4.405446553, 10), #2025: (4.405446553, 4),  # updated to reflect minimum coverage stations
    2026: (14.95639497, 8),
    2027: (32.41860655, 13),
    2028: (56.50379693, 19),
    2029: (86.60130351, 26),
    2030: (122.11487, 33),
    2031: (168.5211193, 42),
    2032: (225.7954578, 52),
    2033: (293.5035729, 64),
    2034: (371.0656292, 76),
    2035: (458.0294513, 89),
    2036: (555.0617497, 102),
    2037: (647.9017273, 114),
    2038: (734.7129444, 125),
    2039: (816.1654149, 135),
    2040: (921.8704233, 147)
}

# Step 5: Remove coverage stations from remaining clusters
remaining_clusters = clusters[~clusters['cluster'].isin(coverage_df['cluster'])].copy()
remaining_clusters = remaining_clusters.sort_values(by='annual_kg_demand', ascending=False)

# Step 6: Create station rollout with demand allocation
kg_per_ton = 1000  # conversion factor
station_rollout = coverage_df.copy()
allocation_records = []

for year, (tons, cumulative_target) in mhdv_rollout.items():
    current_count = len(station_rollout[station_rollout['year_built'] <= year])

    to_add = cumulative_target - current_count
    if to_add > 0:
        additions = remaining_clusters.head(to_add).copy()
        additions = additions.rename(columns={'Lat': 'latitude', 'Long': 'longitude'})
        additions['Name'] = additions['cluster'].apply(lambda x: f"Cluster_{x}")
        additions['year_built'] = year
        station_rollout = pd.concat([
            station_rollout,
            additions[['Name', 'latitude', 'longitude', 'cluster', 'annual_kg_demand', 'year_built']]
        ], ignore_index=True)
        remaining_clusters = remaining_clusters.iloc[to_add:]

    # Allocate hydrogen proportionally to built stations
    active_stations = station_rollout[station_rollout['year_built'] <= year].copy()
    total_demand_kg = tons * kg_per_ton
    total_weight = active_stations['annual_kg_demand'].sum()
    active_stations['hydrogen_kg_allocated_per_day'] = (
        active_stations['annual_kg_demand'] / total_weight * total_demand_kg
    )
    active_stations['year'] = year
    allocation_records.append(active_stations)

# Step 7: Final allocation DataFrame
allocation_df = pd.concat(allocation_records, ignore_index=True)
#allocation_df = allocation_df.drop(columns= ['normalized_radius'], inplace = True)

# Step 8 (Optional): Save results
allocation_df.to_csv("TAZ_hydrogen_demand_2025_2040.csv", index=False)


In [15]:
#allocation_df.drop(columns= ['normalized_radius'], inplace = True)

allocation_df.to_csv("TAZ_hydrogen_demand_2025_2040.csv", index=False)

In [16]:
allocation_df

,Name,latitude,longitude,year_built,cluster,annual_kg_demand,hydrogen_kg_allocated_per_day,year
0,Oakland,37.748749,-122.191906,2025,29,8785.922163,73.772389,2025
1,Fowler,36.605293,-119.658529,2025,142,119166.279613,1000.598567,2025
2,Orland,39.740147,-122.203058,2025,39,95914.994769,805.365466,2025
3,Bakersfield,35.060661,-118.969539,2025,83,52864.702641,443.886860,2025
4,Lost Hills,35.615117,-119.657112,2025,140,33408.501160,280.519779,2025
...,...,...,...,...,...,...,...,...
1052,Cluster_137,32.739821,-116.404206,2040,137,2603.621420,753.311517,2040
1053,Cluster_14,37.495076,-121.324209,2040,14,1218.100255,352.435629,2040
1054,Cluster_78,33.691280,-117.959489,2040,78,998.810665,288.988089,2040
1055,Cluster_61,37.277203,-122.008740,2040,61,913.288005,264.243629,2040


In [25]:
import folium
from folium.plugins import TimestampedGeoJson
import json
import numpy as np

# Example: assuming 'allocation_df' has been loaded from your data source
# Replace with actual path if necessary
# allocation_df = pd.read_csv("yearly_hydrogen_allocation_by_station.csv")

# Min-max scaling for circle sizes
min_radius, max_radius = 4, 20
min_val = allocation_df['hydrogen_kg_allocated_per_day'].min()
max_val = allocation_df['hydrogen_kg_allocated_per_day'].max()

allocation_df['normalized_radius'] = allocation_df['hydrogen_kg_allocated_per_day'].apply(
    lambda x: min_radius + (x - min_val) / (max_val - min_val) * (max_radius - min_radius)
)

# Prepare GeoJSON features for the map
features = []
for _, row in allocation_df.iterrows():
    features.append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'time': f"{row['year']+1}-01-01",
            'popup': f"<b>{row['Name']}</b><br>Year Built: {row['year_built']}<br>Demand: {row['hydrogen_kg_allocated_per_day']:.0f} kg/day",
            'icon': 'circle',
            'iconstyle': {
                'color': 'blue',       # Color of the outline (you can also set it!)
                'weight': 1,           # Thickness of the outline (default is 2, you can make it bigger or smaller)
                'fillColor': 'red',
                'fillOpacity': 0.6,
                'stroke': 'true',
                'radius': row['normalized_radius']
            }
        }
    })

geojson_data = {
    'type': 'FeatureCollection',
    'features': features
}

# Create folium map centered on a specific location
m = folium.Map(location=[36.8, -120.1], zoom_start=6)

# Add the time slider to the map
TimestampedGeoJson(
    geojson_data,
    transition_time=200,
    period='P1Y',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True
).add_to(m)

# Total demand and station count by year (for dynamic labels)
total_demand_by_year = allocation_df.groupby('year')['hydrogen_kg_allocated_per_day'].sum().round().astype(int).to_dict()
station_counts = allocation_df.groupby("year")["Name"].nunique().to_dict()

# Create a div container for dynamic labels
label_html = """
    <div id="hydrogenDemandLabel" 
         style="position: fixed; bottom: 80px; left: 50px; z-index: 9999; 
                background-color: white; padding: 10px; border: 2px solid gray; border-radius: 8px;">
        <b>Year:</b> <span id="yearLabel">2025</span><br>
        <b>Total Demand:</b> <span id="demandLabel">{}</span><br>
        <b>Stations Built:</b> <span id="stationLabel">{}</span>
    </div>
""".format(total_demand_by_year[2025], station_counts[2025])

# Add the label div to the map
#m.get_root().html.add_child(folium.Element(label_html))

# JavaScript to update the labels dynamically based on slider input
label_script = f"""
<script>
    // Store the demand and station counts for each year
    const demandData = {json.dumps(total_demand_by_year)};
    const stationCounts = {json.dumps(station_counts)};

    // Function to update the label information on the map
    function updateHydrogenInfo(year) {{
        document.getElementById('yearLabel').textContent = year;
        document.getElementById('demandLabel').textContent =
            (demandData[year] || 0).toLocaleString() + " kg/day";
        document.getElementById('stationLabel').textContent =
            (stationCounts[year] || 0).toLocaleString();
    }}

    // Wait for the map to load, then hook into the time slider's input event
    setTimeout(() => {{
        const map = document.querySelector('.leaflet-container')._leaflet_map;
        if (map && map.timeDimension) {{
            const slider = document.querySelector('.slider-timestamp');
            if (slider) {{
                // Listen for changes to the slider (input event)
                slider.addEventListener('input', function() {{
                    const year = parseInt(slider.textContent.trim());
                    if (!isNaN(year)) {{
                        updateHydrogenInfo(year);
                    }}
                }});
            }}
        }}
    }}, 500);
</script>
"""

# After creating your map 'm'

min_val = allocation_df['hydrogen_kg_allocated_per_day'].min()
max_val = allocation_df['hydrogen_kg_allocated_per_day'].max()

low_threshold = round(min_val)
medium_threshold = round((min_val + max_val) / 2)
high_threshold = round(max_val)

size_legend_html = """
<div style="
     position: fixed; 
     bottom: 150px; left: 50px; 
     z-index: 9999; 
     background-color: white; 
     padding: 10px; 
     border: 2px solid gray; 
     border-radius: 8px;
     font-size: 14px;
     line-height: 1.2;">
  <b>Dot Size Legend</b><br>
  (kg/day of Hydrogen Demand)<br><br>
  <svg height="140" width="180">
    <circle cx="30" cy="20" r="5" stroke="blue" stroke-width="1" fill="red" fill-opacity="0.6" />
    <text x="50" y="25" font-size="12">0–5,000 kg/day</text>
    
    <circle cx="30" cy="60" r="10" stroke="blue" stroke-width="1" fill="red" fill-opacity="0.6" />
    <text x="50" y="65" font-size="12">5,001–20,000 kg/day</text>
    
    <circle cx="30" cy="100" r="15" stroke="blue" stroke-width="1" fill="red" fill-opacity="0.6" />
    <text x="50" y="105" font-size="12">20,001+ kg/day</text>
  </svg>
</div>
"""


m.get_root().html.add_child(folium.Element(size_legend_html))


# Add dynamic label script to the map
m.get_root().html.add_child(folium.Element(label_script))

# Save final map as an HTML file
final_map_path = "final_hydrogen_station_slider_map_with_dynamic_labels.html"
m.save(final_map_path)
final_map_path


'final_hydrogen_station_slider_map_with_dynamic_labels.html'

In [18]:
print("Years in data:", allocation_df['year'].unique())


Years in data: [2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038
 2039 2040]


In [19]:
allocation_df.groupby('year').count()

,Name,latitude,longitude,year_built,cluster,annual_kg_demand,hydrogen_kg_allocated_per_day,normalized_radius
year,,,,,,,,
2025,10,10,10,10,10,10,10,10
2026,10,10,10,10,10,10,10,10
2027,13,13,13,13,13,13,13,13
2028,19,19,19,19,19,19,19,19
2029,26,26,26,26,26,26,26,26
2030,33,33,33,33,33,33,33,33
2031,42,42,42,42,42,42,42,42
2032,52,52,52,52,52,52,52,52
2033,64,64,64,64,64,64,64,64


In [20]:
allocation_df.groupby('year').sum()

,Name,latitude,longitude,year_built,cluster,annual_kg_demand,hydrogen_kg_allocated_per_day,normalized_radius
year,,,,,,,,
2025,OaklandFowlerOrlandBakersfieldLost HillsMojave...,363.163790,-1197.764710,20250,733,5.246666e+05,4405.446553,41.705676
2026,OaklandFowlerOrlandBakersfieldLost HillsMojave...,363.163790,-1197.764710,20250,733,5.246666e+05,14956.394970,46.612399
2027,OaklandFowlerOrlandBakersfieldLost HillsMojave...,471.816548,-1554.804222,26331,1036,7.720554e+05,32418.606550,66.630285
2028,OaklandFowlerOrlandBakersfieldLost HillsMojave...,690.460224,-2274.978821,38499,1464,1.121878e+06,56503.796930,101.625266
2029,OaklandFowlerOrlandBakersfieldLost HillsMojave...,953.440970,-3121.094312,52702,1950,1.417578e+06,86601.303510,143.381970
2030,OaklandFowlerOrlandBakersfieldLost HillsMojave...,1221.124663,-3966.903056,66912,2197,1.661932e+06,122114.870000,187.657415
2031,OaklandFowlerOrlandBakersfieldLost HillsMojave...,1539.033964,-5038.038212,85191,2903,1.939915e+06,168521.119300,244.929891
2032,OaklandFowlerOrlandBakersfieldLost HillsMojave...,1893.770860,-6228.424201,105511,3580,2.203137e+06,225795.457800,311.222269
2033,OaklandFowlerOrlandBakersfieldLost HillsMojave...,2324.210066,-7664.481574,129907,4500,2.459505e+06,293503.572900,390.298263
